# 1. Text extraction from ontology data


Step 1:  
- Load the ontology
- Understand its classes, properties and possible interconnections

In [1]:
from owlready2 import *

import warnings
warnings.filterwarnings('always')

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
def lookAtTheOntology(onto_path):
    onto = get_ontology(onto_path).load()
    
    '''print('\tClasses: ', *list(cls.name for cls in onto.classes()), sep='\n\t\t')
    print('\n\tIndividuals: ', *list(ind.name for ind in onto.individuals()), sep='\n\t\t')
    print('\n\tProperties: ', *list(prop.name for prop in onto.properties()), sep='\n\t\t')'''

    return onto

onto = lookAtTheOntology('file://../new_FROD_ontology.owl')

FileNotFoundError: [Errno 2] No such file or directory: '../new_FROD_ontology.owl'

_We were told to use a dataset similar to the ontology_  

Performing text extraction using both an ontology and a dataset involves first integrating the dataset into the ontology by mapping the dataset's concepts to the ontology's classes and properties. 

Step 2: (Map dataset to Ontology classes and properties)  
- Create corresponding classes and properties for the dataset
- Create instances of the classes for the dataset

In [7]:
# Load the dataset
import pandas as pd
import numpy as np

banking_path = '../data/Fraud_banking.csv'
csv_banking = pd.read_csv(banking_path)
csv_banking = csv_banking.sample(10000).reset_index(drop=True)

# Convert binary values to categorical values
csv_banking["isFraud_Cat"]        = csv_banking["isFraud"].replace({1: "Fraud", 0: "Not Fraud"})
csv_banking["isFlaggedFraud_Cat"] = csv_banking["isFlaggedFraud"].replace({1: "Yes", 0: "No"})
csv_banking["type_num"]           = csv_banking["type"].replace({'CASH_OUT':1, 'CASH_IN':2, 'PAYMENT':3, 'DEBIT':4, 'TRANSFER':5})

display(csv_banking.head(), csv_banking.shape)

In [ ]:
# Create corresponding classes and properties for the chosen subclass of the ontology 
def addClassestoOntology(ontology, ontology_class, df):
    # Get the list of columns in the dataframe
    columns = list(df.columns)
    # Check if the classes is in the ontology
    if ontology_class in list(ontology.classes()):
        for column in columns:
            with ontology:
                class column(DataProperty):
                    domain  = [ontology_class]
                    range = [df[column].dtype]
    # Check if the classes and properties have been created
    # --------------------------------------------------- Not sure how to do this yet 
    
    return ontology;

onto = addClassestoOntology(onto, 'Transaction', csv_banking)

In [ ]:
# Create an entry key, value pair for each attribute in every row of the dataset
def create_entry(df,index,columns=csv_banking.columns[1:]):
    entry = {}
    for x,y in zip(columns,df.iloc[index][columns]):
        if type(y)==np.float64:
            y = float(y)
        elif type(y)==np.int64:
            y=int(y)
    
        entry[x]=[y] 
    if 'zip' in entry:
        entry['_zip']=entry['zip']
        del entry['zip']
    #print('Entry: ',entry, sep='\t')
    return entry

# Create instances of the classes for each row of the dataset
def createInstances(ontology, dataset, chosenClass):
    with ontology:
        for idx,_ in dataset.iterrows():
            row_entry = create_entry(dataset, idx)
            transaction = ontology.chosenClass(chosenClass+'_'+str(idx), **row_entry)
            # print('Transaction: ', transaction, 'Entry', row_entry, sep='\t')
    # Check if the instances have been created
    print(len(list(onto.Transaction.instances())), ' Instances created ', sep='\t')
    return ontology;

onto_n = createInstances(onto, csv_banking, 'Transaction')

10000	 Instances created 


# 2. Using extracted text to create rules for fraud detection


Step :  
- define extraction rules based on the client's specifications

I want to focus on the Transaction class   

_The client has chosen the classes to evaluate so from here on our data is irrelevant_

In [ ]:
listOfScenarios = ['Identity_Theft','Credit_Card','Investment','Banking'] # The list of classes are the different fraud scenarios

In [ ]:
def IdentityTheftRules(ontology): # Validate the identity Theft scenario with the rule statement ans send a notification to the user
    # check what subclass of the ontology you are working with
    if ontology.classes() == [ontology.Identity_Theft]:
        # Create the rule statement
        rule_statement1 = 'for x in ontology.Identity_Theft.instances():\
            if x.issued == "France" and x.ATM_Withdrawal_location_timezone == "United States":\
                print("Identity Theft")'
        rule_statement2 = 'for x in ontology.Identity_Theft.instances():\
            if x.issued == "France" and x.ATM_Withdrawal_location_timezone == "Europe":\
                pass'
        rule_statement3 = 'for x in ontology.Identity_Theft.instances():\
            if x.subscription_payement in ["RATP","SNCF","SFR","FREEMOBILE","LYCAMOBILE"]:\
                print("Identity Theft")'
        rule_statement4 = 'for x in ontology.Identity_Theft.instances():\
            if x.insurance in ["Orange","Bouygues","AXA"]:\
                pass'
        rule_statement5 = 'for x in ontology.Identity_Theft.instances():\
            if x.wrong_transaction_pin > 3:\
                print("Identity Theft , Blocked Account")'
        rule_statement6 = 'for x in ontology.Identity_Theft.instances():\
            if x.Account_holder_name == "Adamu Hussaini" and x.share_purchase == "Adamu Hussaini":\
                pass'
        # Execute the rule statement
        exec(rule_statement1)
        exec(rule_statement2)
        exec(rule_statement3)
        exec(rule_statement4)
        exec(rule_statement5)
        exec(rule_statement6)
    return 

def CreditCardRules(ontology): # Validate the credit card scenario with the rule statement ans send a notification to the user
    # check what subclass of the ontology you are working with
    if ontology.classes() == [ontology.Credit_Card]:
        # Create the rule statements
        rule_statement1 = 'for x in ontology.Credit_Card.instances():\
            if x.IP_address == "18970988" and x.geolocation == "Paris":\
                pass'
        rule_statement2 = 'for x in ontology.Credit_Card.instances():\
            if x.IP_address == "18970988" and x.geolocation != "Paris":\
                print("Credit Card Fraud")'
        rule_statement3 = 'for x in ontology.Credit_Card.instances():\
            if x.Number_of_transactions > 3:\
                print("Credit Card Fraud")'
        rule_statement4 = 'for x in ontology.Credit_Card.instances():\
            if x.transaction_amount > 1000:\
                print("Credit Card Fraud")'
        rule_statement5 = 'for x in ontology.Credit_Card.instances():\
            if x.Account_holder_name == "Adamu Hussaini" and x.share_purchase != "Adamu Hussaini":\
                print("Identity Theft")'
        rule_statement6 = 'for x in ontology.Credit_Card.instances():\
            if x.online_payment_platform_address == "France" and x.online_payment_platform_address == "Euro zone":\
                pass'
        rule_statement7 = 'for x in ontology.Credit_Card.instances():\
            if x.online_payment_platform_address in ["Nigeria","Asia","North America"]:\
                print("Credit Card Fraud")'
        rule_statement8 = 'for x in ontology.Credit_Card.instances():\
            if x.device_hash == "Hidden":\
                print("Credit Card Fraud")'
        # Execute the rule statement
        exec(rule_statement1)
        exec(rule_statement2)
        exec(rule_statement3)
        exec(rule_statement4)
        exec(rule_statement5)
        exec(rule_statement6)
        exec(rule_statement7)
        exec(rule_statement8)
    return
        
def InvestmentRules(ontology): # Validate the investment scenario with the rule statement ans send a notification to the user
    # check what subclass of the ontology you are working with
    if ontology.classes() == [ontology.Investment]:
        # Create the rule statements
        rule_statement1 = 'for x in ontology.Investment.instances():\
            if x.Transaction == "investment payment" and x.payment_gateway == "www.sharepurchase.com":\
                print("Investment Fraud")'
        rule_statement2 = 'for x in ontology.Investment.instances():\
            if x.Transaction == "investment scheme":\
                print("Investment Fraud, Blocked")'
        rule_statement3 = 'for x in ontology.Investment.instances():\
            if x.Transaction_amount < 1000:\
                pass'
        rule_statement4 = 'for x in ontology.Investment.instances():\
            if x.previous_transaction("Adamu Hussaini") == "charge back":\
                print("Investment Fraud")'

# 3. Test the rules


Step 1:  
- Validate against the ontology.  

In [ ]:
IdentityTheftRules(onto_n)
CreditCardRules(onto_n)
InvestmentRules(onto_n)

_Can't validate without appropriate data_   

With the code, hopefully we have verified the extraction rules for the individuals.  

# 4. Take away

Lessons learnt from working on the project:
- Working with ontology data is crazy fun ... emphasis on __crazy__.  
- When we break things down, the tasks become a whole less complicated and daunting.
- It is important to write code in a modular format